In [1]:
import os
import pandas as pd
from datetime import time

In [2]:
def filter_eligible_trips(df):
  eligible_trips = pd.DataFrame()
  max_stop_sequence = df.loc[df['stop_sequence'].idxmax()]["stop_sequence"]
  df['arrival_time_iso'] = pd.to_datetime(df['arrival_time'], unit='s')
  df['arrival_time_iso'] = df['arrival_time_iso'].dt.tz_localize('UTC')
  df['arrival_time_local'] = df['arrival_time_iso'].dt.tz_convert('Europe/Stockholm')

  for trip_id, trip_group in df.groupby("trip_id"):
    first_stop = trip_group[trip_group['stop_sequence'] == 1]
    last_stop = trip_group[trip_group['stop_sequence'] == max_stop_sequence]

    if first_stop.shape[0] == 0 or last_stop.shape[0] == 0:
      print(f"Trip {trip_id} missing first or last stop", first_stop.shape[0], last_stop.shape[0])
      continue

    first_arrival = first_stop["arrival_time_local"].iloc[0] 
    last_arrival = last_stop["arrival_time_local"].iloc[0]
      
    first_cutoff = first_arrival.replace(hour=6, minute=30, second=0, microsecond=0)
    last_cutoff = last_arrival.replace(hour=9, minute=0, second=0, microsecond=0)

    if first_arrival > first_cutoff and last_arrival < last_cutoff:
      eligible_trips = pd.concat([eligible_trips, trip_group])
   
  return eligible_trips



def concatenate_all_raw(type_day_str):
    trips_snow_raw = []

    for filename in os.listdir("./../raw_data/"+type_day_str):
        print(filename)
        if filename.split("-")[3] == "arrivals":
            df_s = pd.read_csv(f"./../raw_data/{type_day_str}/{filename}")
            df_s['date'] = pd.to_datetime(df_s['arrival_time'], unit='s').dt.date
            trips_snow_raw.append(filter_eligible_trips(df_s))

    df_all_snow = pd.concat(trips_snow_raw, ignore_index=True)
    df_all_snow['trip_day_id'] = df_all_snow.groupby(['trip_id', 'date']).ngroup()
    
    return df_all_snow



def process_trip_groups(df):
    results = []
    missing_stop_dict = {} 

    grouped = df.groupby('trip_day_id')

    for trip_id, group in grouped:

        # get all unique busstops in df ('stop_sequence' or 'stop_id')
        unique_stops = sorted(group['stop_sequence'].unique())
        missing_stops = set(range(1, group['stop_sequence'].max())) - set(unique_stops)

        # add trip_id to proper bucket based on which stops are missing
        key = frozenset(missing_stops)
        if key not in missing_stop_dict:
            missing_stop_dict[key] = []
        missing_stop_dict[key].append(trip_id)

        # get all certain stops
        # arrival-wise
        certain_arriv = group[
            group.arrival_uncertainty == 0.0
        ].drop_duplicates(subset=['stop_sequence'], keep='first').sort_values(by='stop_sequence')

        # departure-wise
        certain_depart = group[
            (group.arrival_uncertainty != 0.0) & (group.departure_uncertainty == 0.0)
        ].drop_duplicates(subset=['stop_sequence'], keep='first').sort_values(by='stop_sequence')

        # find stops for which no certain info
        certain_stops = set(certain_arriv['stop_sequence']).union(set(certain_depart['stop_sequence']))
        uncertain_stops = set(unique_stops) - certain_stops

        # get best estimates (most recent)
        best_est = group[group['stop_sequence'].isin(uncertain_stops)].sort_values(by='update_timestamp', ascending=False).drop_duplicates(subset='stop_sequence', keep='first')

        # combine
        combined = pd.concat([certain_arriv, certain_depart, best_est], ignore_index=True).drop_duplicates(subset='stop_sequence', keep='first').sort_values(by='stop_sequence')

        results.append(combined)

    return pd.concat(results), missing_stop_dict

In [3]:
# get all raw_data & filter by time
trips_snow = concatenate_all_raw('snow_days')
trips_control = concatenate_all_raw('control_days')
trips_second_pull = concatenate_all_raw('second_pull')
trips_additional_control = concatenate_all_raw('additional_control')

2024-02-13-missing-arrivals.csv
2023-02-20-arrivals-and-departures.csv
Trip 14010000620087893 missing first or last stop 46 0
Trip 14010000620088135 missing first or last stop 53 0
Trip 14010000623434270 missing first or last stop 0 22
Trip 14010000623434345 missing first or last stop 0 122
Trip 14010000623434984 missing first or last stop 56 0
Trip 14010000623752130 missing first or last stop 0 54
Trip 14010000623752222 missing first or last stop 0 96
2023-12-05-arrivals-and-departures.csv
Trip 14010000635912515 missing first or last stop 0 45
Trip 14010000635912781 missing first or last stop 0 128
Trip 14010000635915098 missing first or last stop 55 0
Trip 14010000635915829 missing first or last stop 42 0
Trip 14010000641238916 missing first or last stop 0 45
Trip 14010000641239191 missing first or last stop 0 93
Trip 14010000641241623 missing first or last stop 53 0
2022-02-08-arrivals-and-departures.csv
Trip 14010000592177356 missing first or last stop 0 27
Trip 14010000592177688 m

In [4]:
dedup_snow, _ = process_trip_groups(trips_snow)
dedup_control, _ = process_trip_groups(trips_control)
dedup_second_pull, _ = process_trip_groups(trips_second_pull)
dedup_additional_control, _ = process_trip_groups(trips_additional_control)

In [7]:
weather_control = pd.read_csv('./control_days_DEPTH_BASED.csv')
weather_snow = pd.read_csv('./snow_days_DEPTH_BASED.csv')
weather_combined = pd.concat([weather_control, weather_snow], ignore_index=True)

trips_combined = pd.concat([dedup_snow, dedup_control, dedup_second_pull, dedup_additional_control], ignore_index=True)

trips_combined['date'] = pd.to_datetime(trips_combined['date'])
weather_combined['date'] = pd.to_datetime(weather_combined['date'])

all_trip_data = trips_combined[trips_combined['date'].isin(weather_combined['date'])]

In [8]:
all_trip_data

,trip_id,stop_sequence,stop_id,arrival_time,arrival_delay,arrival_uncertainty,departure_time,departure_delay,departure_uncertainty,update_timestamp,trip_schedule_relationship,date,arrival_time_iso,arrival_time_local,trip_day_id
555,14010000572359051,1,9022001010098003,1610602254,54,NaN,1610602254,54,0.0,1610602789,0,2021-01-14,2021-01-14 05:30:54+00:00,2021-01-14 06:30:54+01:00,18
556,14010000572359051,2,9022001010627001,1610602337,78,NaN,1610602340,81,NaN,1610602849,0,2021-01-14,2021-01-14 05:32:17+00:00,2021-01-14 06:32:17+01:00,18
557,14010000572359051,3,9022001010203002,1610602373,74,0.0,1610602373,74,0.0,1610602789,0,2021-01-14,2021-01-14 05:32:53+00:00,2021-01-14 06:32:53+01:00,18
558,14010000572359051,4,9022001010045001,1610602488,137,0.0,1610602488,137,0.0,1610602789,0,2021-01-14,2021-01-14 05:34:48+00:00,2021-01-14 06:34:48+01:00,18
559,14010000572359051,5,9022001010040001,1610602608,185,0.0,1610602608,185,0.0,1610602789,0,2021-01-14,2021-01-14 05:36:48+00:00,2021-01-14 06:36:48+01:00,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120788,14010000649240506,21,9022001010198002,1707119585,22,0.0,1707119605,42,0.0,1707119856,0,2024-02-05,2024-02-05 07:53:05+00:00,2024-02-05 08:53:05+01:00,858
120789,14010000649240506,22,9022001010045002,1707119773,-10,0.0,1707119773,-10,0.0,1707119856,0,2024-02-05,2024-02-05 07:56:13+00:00,2024-02-05 08:56:13+01:00,858
120790,14010000649240506,23,9022001010203001,1707119837,-15,0.0,1707119848,-4,0.0,1707119856,0,2024-02-05,2024-02-05 07:57:17+00:00,2024-02-05 08:57:17+01:00,858
120791,14010000649240506,24,9022001010627002,1707119925,-21,0.0,1707119943,-3,0.0,1707119951,0,2024-02-05,2024-02-05 07:58:45+00:00,2024-02-05 08:58:45+01:00,858


In [ ]:
all_trip_data.to_csv("./all_trip_data.csv", index=False)